In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# rodar o google drive no colab, puxa todo conteúdo do drive da pessoa
from google.colab import drive 
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TrabalhoPratico").getOrCreate()

title_basics = spark.read \
    .format('csv') \
    .option("sep", "	") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/content/drive/MyDrive/MBA-mod2/title_basics.tsv")

title_basics.show(20)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [19]:
title_basics.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



In [3]:
title_ratings = spark.read \
    .format('csv') \
    .option("sep", "	") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/content/drive/MyDrive/MBA-mod2/title_ratings.tsv")

title_ratings.show(20)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    1809|
|tt0000002|          6.0|     233|
|tt0000003|          6.5|    1560|
|tt0000004|          6.1|     152|
|tt0000005|          6.2|    2383|
|tt0000006|          5.1|     157|
|tt0000007|          5.4|     746|
|tt0000008|          5.5|    1965|
|tt0000009|          5.8|     189|
|tt0000010|          6.9|    6530|
|tt0000011|          5.2|     323|
|tt0000012|          7.4|   11188|
|tt0000013|          5.8|    1726|
|tt0000014|          7.1|    5023|
|tt0000015|          6.2|     947|
|tt0000016|          5.9|    1325|
|tt0000017|          4.6|     292|
|tt0000018|          5.3|     537|
|tt0000019|          5.3|      28|
|tt0000020|          5.0|     315|
+---------+-------------+--------+
only showing top 20 rows



In [22]:
title_ratings.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)



In [26]:
title_basics.groupBy("titleType").count().toPandas()

,titleType,count
0,tvSeries,213405
1,tvMiniSeries,39133
2,tvMovie,132690
3,tvEpisode,6012696
4,movie,585950
5,tvSpecial,33896
6,video,319124
7,videoGame,28962
8,tvShort,10236
9,short,827595


In [27]:
title_basics.filter("startYear == '2015' and titleType in ('tvMovie', 'movie')").count()

19987

In [5]:
import pyspark.sql.functions as f

In [8]:
title_array = title_basics.withColumn('genres_array', f.split(f.col('genres'), ','))
title_array.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|        genres_array|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|[Documentary, Short]|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|  [Animation, Short]|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|[Animation, Comed...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation

In [9]:
title_exp = title_array.withColumn('genres_unico', f.explode(f.col('genres_array')))
title_exp.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+--------------------+------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|        genres_array|genres_unico|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+--------------------+------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|[Documentary, Short]| Documentary|
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|[Documentary, Short]|       Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|  [Animation, Short]|   Animation|
|tt0000002|    short|Le clown et ses c..

In [10]:
title_exp.groupBy("genres_array").count().orderBy("count").toPandas()

,genres_array,count
0,"[Film-Noir, Horror]",2
1,"[Adventure, Film-Noir]",2
2,"[Musical, Sport]",2
3,"[Talk-Show, War]",2
4,"[News, Romance]",2
...,...,...
2285,[Talk-Show],467788
2286,[Comedy],568956
2287,[\N],643012
2288,"[Drama, Romance]",709470


In [11]:
df_left = title_ratings.join(title_exp, "tconst", "left")

df_left.show(5)

+---------+-------------+--------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+--------------------+------------+
|   tconst|averageRating|numVotes|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|           genres|        genres_array|genres_unico|
+---------+-------------+--------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+--------------------+------------+
|tt1790891|          6.5|       6|    short|The Cameras of Va...|Vaalan kamerat py...|      0|     1937|     \N|             8|Documentary,Short|[Documentary, Short]| Documentary|
|tt1790891|          6.5|       6|    short|The Cameras of Va...|Vaalan kamerat py...|      0|     1937|     \N|             8|Documentary,Short|[Documentary, Short]|       Short|
|tt0000002|          6.0|     233|    short|Le clown et ses c...|Le clown et ses c...|      0|     1

In [12]:
df_left.agg(f.max("averageRating").alias('nota_maxima')).show(20)

+-----------+
|nota_maxima|
+-----------+
|       10.0|
+-----------+



In [13]:
df_left.filter("averageRating == '10'").groupBy("genres_unico").count().toPandas()

,genres_unico,count
0,Crime,92
1,Romance,176
2,Thriller,29
3,Adventure,382
4,\N,53
5,Drama,2099
6,War,3
7,Documentary,383
8,Reality-TV,133
9,Family,297


In [17]:
df_left.filter("titleType == 'videoGame' and startYear=='2020' and averageRating >= '9.5'").toPandas()

,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_array,genres_unico
0,tt11321196,9.5,506,videoGame,Half-Life: Alyx,Half-Life: Alyx,0,2020,\N,\N,"Action,Adventure,Horror","[Action, Adventure, Horror]",Action
1,tt11321196,9.5,506,videoGame,Half-Life: Alyx,Half-Life: Alyx,0,2020,\N,\N,"Action,Adventure,Horror","[Action, Adventure, Horror]",Adventure
2,tt11321196,9.5,506,videoGame,Half-Life: Alyx,Half-Life: Alyx,0,2020,\N,\N,"Action,Adventure,Horror","[Action, Adventure, Horror]",Horror


In [18]:
from pyspark.sql.window import Window

In [24]:
df_titles_subset = (
    df_left
    .filter("cast(startYear as int) == 2018")
    .withColumn('genre', f.split('genres', ',').getItem(0))
)

In [25]:
df_titles_subset.count()

107080

In [21]:
df_titles_subset.withColumn('genre', f.split('genres', ',').getItem(0)).limit(5).toPandas()

,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_array,genres_unico,genre
0,tt0137818,4.4,31,movie,Housesitter: The Night They Saved Siegfried's ...,Housesitter: The Night They Saved Siegfried's ...,0,2018,\N,95,\N,[\N],\N,\N
1,tt0170651,6.8,14,movie,TGM the Liberator,T.G.M. - osvoboditel,0,2018,\N,60,Documentary,[Documentary],Documentary,Documentary
2,tt0271577,5.0,6,short,Je suis votre voisin,Je suis votre voisin,0,2018,\N,21,Short,[Short],Short,Short
3,tt0328810,7.6,85,movie,My Land,Aamaar Bhuvan,0,2018,\N,102,"Drama,Family","[Drama, Family]",Drama,Drama
4,tt0328810,7.6,85,movie,My Land,Aamaar Bhuvan,0,2018,\N,102,"Drama,Family","[Drama, Family]",Family,Drama


In [34]:
w = Window.partitionBy('genre').orderBy('startYear')
(
    df_titles_subset
    .withColumn('genre', f.split('genres', ',').getItem(0))
    .withColumn('startYear', f.col('startYear').cast('int'))
    .filter('startYear >= 2018 and genres_unico == "Comedy"')
    .withColumn('rn', f.percent_rank().over(w))
    .limit(25)
    .toPandas()
)

,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_array,genres_unico,genre,rn
0,tt10140580,7.4,5,short,Draugurinn - A Ghostbusters fan film,Draugurinn - A Ghostbusters fan film,0,2018,\N,5,"Action,Comedy,Short","[Action, Comedy, Short]",Comedy,Action,0.0
1,tt10317978,6.8,146,tvMovie,Mob Psycho 100 REIGEN - The Miracle Psychic th...,Mob Psycho 100 REIGEN - The Miracle Psychic th...,0,2018,\N,60,"Action,Animation,Comedy","[Action, Animation, Comedy]",Comedy,Action,0.0
2,tt10384130,4.2,5,movie,Fight Back to School,Truong Hoc Ba Vuong,0,2018,\N,99,"Action,Comedy","[Action, Comedy]",Comedy,Action,0.0
3,tt10683490,6.0,5,tvEpisode,Change the World,Change the World,0,2018,\N,24,"Action,Animation,Comedy","[Action, Animation, Comedy]",Comedy,Action,0.0
4,tt10683538,5.8,5,tvEpisode,It's Showtime!!,It's Showtime!!,0,2018,\N,24,"Action,Animation,Comedy","[Action, Animation, Comedy]",Comedy,Action,0.0
5,tt10683558,6.0,5,tvEpisode,Be Honest,Be Honest,0,2018,\N,24,"Action,Animation,Comedy","[Action, Animation, Comedy]",Comedy,Action,0.0
6,tt10683582,6.0,5,tvEpisode,Shinovi Versus,Shinovi Versus,0,2018,\N,24,"Action,Animation,Comedy","[Action, Animation, Comedy]",Comedy,Action,0.0
7,tt10683704,5.6,5,tvEpisode,Two Hearts,Two Hearts,0,2018,\N,24,"Action,Animation,Comedy","[Action, Animation, Comedy]",Comedy,Action,0.0
8,tt10684910,5.8,5,tvEpisode,Hometown,Hometown,0,2018,\N,24,"Action,Animation,Comedy","[Action, Animation, Comedy]",Comedy,Action,0.0
9,tt10709388,7.7,20,short,Anime: The Fast & The Furious,Anime: The Fast & The Furious,0,2018,\N,7,"Action,Comedy,Short","[Action, Comedy, Short]",Comedy,Action,0.0


In [27]:
df_left.filter("startYear == '2018'").count()

107080

In [31]:
df_left.filter("startYear == '2018' and genres_unico == 'Comedy'").count()

16000